## Modely na spracovanie signálov z vibrácií

Modely na monitorovanie vibrácií na základe analyzovaných algoritmov detekcie špičiek. Meranie úspešnosti na synteticky generovaných dátach.

### Toky v návrhu riešenia
1. Frekvenčná analýza
2. Online štatistiky v čase
3. Podvzorkovanie

#### Algoritmy na rozpoznávanie špičiek
za aplikovania vyhladzovania cez priemerovací filter

0. Detekcia špičiek prahovou úrovňou
1. Význačnosť vrchola spomedzi susedov
2. Algoritmus prechodu nulou do záporu
3. Algoritmus horského turistu

#### Rozklad vo frekvenčnej doméne
- Fouriérová transformácia - FFT
- DCT-II, DCT-IV
- Oknové funkcie: obdĺžníkové, Bartlettovo, Hannovo, Hammingovo a Blackmanovo okno
- Welchova metóda

In [ ]:
import numpy as np
from scipy.signal.windows import (
    boxcar,
    bartlett,
    hann,
    hamming,
    blackman,
    gaussian
)
import matplotlib.pyplot as plt
from functools import partial
from pprint import pprint

from signals.signal_generator import *
from signals.event_detection import *
from signals.metrics import *
from signals.graphs import *
from signals.loader import *

gauss = partial(gaussian, std=1)

### Generovanie sínusoidy
s frekvenciou, trvaním, pri vzorkovacej frekvencii

In [ ]:
x, y = sine_generate(f=10, t=1, fs=200)
fig, axes = plt.subplots(3, 1, figsize=(20, 10))

p = find_peaks_neighbours(y, k=2, e=0, h_rel=0, h=0)
print('find_peaks_neighbours:\t', p)
plot_sinusoid(x, y, p, axes[0])

p = find_peaks_zero_crossing(y, k=1, slope=0)
print('find_peaks_zero_crossing:', p)
plot_sinusoid(x, y, p, axes[1])

p = find_peaks_hill_walker(y, tolerance=0, hole=0, prominence=0, isolation=0)
print('find_peaks_hill_walker:\t', p)
plot_sinusoid(x, y, p, axes[2])

### Prepočet poradia frekvenčného vedierka na frekvencie v Hz pre skutočné a predikované špičky

In [ ]:
def found_peak_freqs(truth, peaks, spectrum):
    f_truth = peak_freqs_windowed(truth, spectrum['bins'], spectrum['resolution'])
    f_peaks = peak_freqs_windowed(peaks, spectrum['bins'], spectrum['resolution'])

    for t_win, p_win in zip(f_truth, f_peaks):
        print("Okno:")
        print('\tI:', ', '.join([f'<{a:.2f}-{b:.2f}>' for a, b in t_win]))
        print('\tO:', ', '.join([f'<{a:.2f}-{b:.2f}>' for a, b in p_win]))  

### Generovanie sinusoidy s f = 15 Hz, trvaním t = 1s pri fs = 250Hz

In [ ]:
plt.rcParams['figure.figsize'] = (10, 3)
rec = {
    'duration': 1,
    'f_sampling': 250,
    'sines': [{'freq': 15, 'amp': 1, 't0': 0, 'tn': 1}],
}
t, y = sine_generate(f=rec['sines'][0]['freq'], t=rec['duration'], fs=rec['f_sampling'])
plt.plot(t, y)
plt.show()

### Hľadanie špičiek spomedzi najbližších susedov vo frekvenčnej doméne na sínusoide s f = 20 Hz

In [ ]:
spectrum = frequency_spectrum(t, y, fs=rec['f_sampling'], window=hann, Wn=63, overlap=0, mode='linear')
func = partial(find_peaks_neighbours, k=3, e=0, h_rel=0, h=-100)

peaks = peaks_indices_windowed(func, spectrum['magnitudes'])
plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

truth = ground_truth(rec, spectrum['t_windows'], spectrum['Wn'], spectrum['resolution'])
found_peak_freqs(truth, peaks, spectrum)

### Vyhodnotenie klasifikácie špičiek

In [ ]:
m = metrics(spectrum['bins'], truth, peaks)

print('Podľa okna:')
pprint(m)
print('Makro:', metric_macro_average(m))
print('Mikro:', metric_micro_average(m))

### Hľadanie špičiek spomedzi najbližších susedov vo frekvenčnej doméne 
sínusoida s f = 8 Hz, t = 4 s, fs = 100 Hz v logaritmickej mierke

In [ ]:
rec = {
    'duration': 4,
    'f_sampling': 100,
    'sines': [{'freq': 8, 'amp': 1, 't0': 0, 'tn': 4}],
}
t, y = sine_generate(f=rec['sines'][0]['freq'], t=rec['duration'], fs=rec['f_sampling'])

spectrum = frequency_spectrum(t, y, fs=rec['f_sampling'], window=hann, Wn=127, overlap=0.25, mode='log')
peaks = peaks_indices_windowed(
    peak_find=partial(find_peaks_neighbours, k=3, e=0, h_rel=0, h=-100),
    magnitudes=spectrum['magnitudes']
)
plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

truth = ground_truth(rec, spectrum['t_windows'], spectrum['Wn'], spectrum['resolution'])
found_peak_freqs(truth, peaks, spectrum)

### Generovanie signálu s normálne rozdeleným šumom

In [ ]:
plt.rcParams['figure.figsize'] = (10, 3)
y_noisy = y + np.random.normal(loc=0, scale=0.2, size=len(y))
plt.plot(t, y_noisy)
plt.grid()
plt.show()

### Generátor syntetického signálu so šumom zo sinusioíd
Pravidlá na opis požadovaného signálu musia dodržať nasledujúcu štruktúru:
```json
{
    "duration": 1,
    "f_sampling": 476,
    "sines": [
        {"freq": 5, "amp": 1.5, "t0": 0, "tn": 0.5}
    ],
    "noises": [
        {"amp": 0.2, "t0": 0, "tn": 1.5}
    ]
}
```

In [ ]:
# Prípadne načítanie pravidiel zo súboru
recording = signal_rules('datasets/synth-signal/rec.json')
print(recording)

# Pravidlá na poskladanie tónov
recording = {
    'duration': 5,
    'f_sampling': 476,
    'sines': [
        {'freq': 5, 'amp': 1.5, 't0': 0, 'tn': 1.5},
        {'freq': 18, 'amp': 1.5, 't0': 1.2, 'tn': 5},
        {'freq': 30, 'amp': 1, 't0': 0, 'tn': 5},
        {'freq': 75, 'amp': 2.8, 't0': 0, 'tn': 5},
        {'freq': 230, 'amp': 0.5, 't0': 1, 'tn': 5}
    ],
    'noises': [
        {'amp': 0.2, 't0': 0, 'tn': 5}
    ]
}

# Vzorkovaný signál podľa popisu
t, y = signal_generate(recording)
# Ponechanie vzoriek nad prahovú magnitúdu a vynulovanie ostatných
y_event = signal_threshold(y, 0.8)
# Vyhladenie signálu so zvolenou okovou funkciou a veľkosťou kernelu
t_s, y_s = smooth_signal(t, y_event, boxcar, k=10)

# Vykreslenie grafu
plt.rcParams['figure.figsize'] = (20, 10)
fig, axes = plt.subplots(3, 1)
axes[0].set_title('Syntetický signál so šumom')
axes[0].set_ylabel('Amplitúda')
axes[0].plot(t, y)
axes[0].grid()

axes[1].set_title('Ponechanie najvyšších magnitúd y > 0.8')
axes[1].plot(t, y_event)
axes[1].set_ylabel('Amplitúda')
axes[1].grid()

axes[2].set_title('Vyhladenie udalostí')
axes[2].plot(t_s, y_s)
axes[2].set_ylabel('Amplitúda')
axes[2].set_xlabel('Čas [s]')
axes[2].grid()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (20, 5)
fig, axes = plt.subplots(1, 3)
axes[0].hist(y, bins=100)
axes[1].hist(y_event, bins=100)
axes[2].hist(y_s, bins=100)
plt.show()

### Hľadanie vrcholov v jednom spektrograme. Podklad pre unit test na zhodu implementácii v C a Pythone

In [ ]:
recording = {
    'duration': 3,
    'f_sampling': 32,
    'sines': [
        {'freq': 2, 'amp': 2, 't0': 0, 'tn': 3},
        {'freq': 6, 'amp': 1, 't0': 0, 'tn': 3},
        {'freq': 7.5, 'amp': 0.7, 't0': 0, 'tn': 3},
        {'freq': 8, 'amp': 1.5, 't0': 0, 'tn': 3},
        {'freq': 12, 'amp': 3, 't0': 0, 'tn': 3}
    ],
    'noises': [
        {'amp': 0.4, 't0': 0, 'tn': 3}  # gaussian noise
    ]
}

np.random.seed(1)
t, y = signal_generate(recording)
# y += 10  # DC offset

#plt.plot(t, y)
spectrum = frequency_spectrum(t, y, fs=recording['f_sampling'], window=hamming, Wn=64, overlap=0.5, mode='log')

tf = spectrum['bins']
yf = spectrum['magnitudes'][0]

fig, ax = plt.subplots(3, 1, figsize=(20, 10))

pA = find_peaks_neighbours(yf, k=4, e=0, h_rel=10, h=-100)
plot_sinusoid(tf, yf, pA, ax[0])
pB = find_peaks_zero_crossing(yf, k=2, slope=7)
plot_sinusoid(tf, yf, pB, ax[1])
pC = find_peaks_hill_walker(yf, tolerance=1, hole=0, prominence=8, isolation=0)
plot_sinusoid(tf, yf, pC, ax[2])

# Print test data for unit test in C
print(len(yf))
print(list(yf))
pp = np.full(len(yf), 0); pp[pA] = 1
print(list(pp))
pp = np.full(len(yf), 0); pp[pB] = 1
print(list(pp))
pp = np.full(len(yf), 0); pp[pC] = 1
print(list(pp))

### Detekcia špičiek v signále so šumom manuálnym ladením hyperparametrov

In [ ]:
recording = {
    'duration': 5,
    'f_sampling': 476,
    'sines': [
        {'freq': 5, 'amp': 1.5, 't0': 0, 'tn': 1.5},
        {'freq': 18, 'amp': 1.5, 't0': 1.2, 'tn': 5},
        {'freq': 30, 'amp': 1, 't0': 0, 'tn': 5},
        {'freq': 75, 'amp': 2.8, 't0': 0, 'tn': 5},
        {'freq': 230, 'amp': 0.5, 't0': 1, 'tn': 5}
    ],
    'noises': [
        {'amp': 0.2, 't0': 0, 'tn': 5}
    ]
}

t, y = signal_generate(recording)
y = signal_threshold(y, 0.8) 

spectrum = frequency_spectrum(t, y, fs=recording['f_sampling'], window=hann, Wn=511, overlap=0.5, mode='log')
smooth_spectra(spectrum, boxcar, 4)

func = partial(find_peaks_neighbours, k=9, e=0, h_rel=15, h=-100)
# func = partial(find_peaks_zero_crossing, k=2, slope=4)
# func = partial(find_peaks_hill_walker, tolerance=0, hole=0, prominence=10, isolation=5)

peaks = peaks_indices_windowed(func, spectrum['magnitudes'])
plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

truth = ground_truth(recording, spectrum['t_windows'], spectrum['Wn'], spectrum['resolution'])
m = metrics(spectrum['bins'], truth, peaks)

print('Macro:', metric_macro_average(m))
print('Micro:', metric_micro_average(m))

### Detektor udalostí testovaný na syntetických dátach

In [ ]:
events = event_detector(spectrum['bins'], spectrum['magnitudes'], peaks, min_duration=2, time_proximity=2)
events

### Vizualizácia udalostí v spektrograme

In [ ]:
plot_spectra_events(spectrum, events)

### Úspešnosť klasifikátora na trénovacej a testovacej množine
Spôsob hľadania najlepších parametrov je grid search na trénovacich dátach

In [ ]:
rec_train = {
    'duration': 30,
    'f_sampling': 128,
    'sines': [
        {'freq': 5, 'amp': 1.8, 't0': 0, 'tn': 10},
        {'freq': 12, 'amp': 2.3, 't0': 5, 'tn': 25},
        {'freq': 45, 'amp': 0.8, 't0': 0, 'tn': 30},
    ],
    'noises': [
        {'amp': 0.1, 't0': 0, 'tn': 5}
    ]
}
rec_test = {
    'duration': 8,
    'f_sampling': 128,
    'sines': [
        {'freq': 8, 'amp': 1.6, 't0': 5, 'tn': 8},
        {'freq': 24, 'amp': 2.7, 't0': 0, 'tn': 6},
        {'freq': 32, 'amp': 1.1, 't0': 0, 'tn': 8},
    ],
    'noises': [
        {'amp': 0.1, 't0': 0, 'tn': 5}
    ]
}
t_train, y_train = signal_generate(rec_train)
t_test, y_test = signal_generate(rec_test)

train_spectra = frequency_spectrum(
    t_train, y_train, fs=rec_train['f_sampling'], window=hann, Wn=255, overlap=0.5, mode='linear'
)
test_spectra = frequency_spectrum(
    t_test, y_test, fs=rec_test['f_sampling'], window=hann, Wn=255, overlap=0.5, mode='linear'
)

train_labels = ground_truth(
    rec_train, train_spectra['t_windows'], 255, train_spectra['resolution']
)
test_labels = ground_truth(
    rec_test, test_spectra['t_windows'], 255, test_spectra['resolution']
)

### Úspešnosť: význačnosť vrchola spomedzi susedov

In [ ]:
def algo_no1(spectrum, k, e, h_rel):
    f = partial(find_peaks_neighbours, k=int(k), e=int(e), h_rel=h_rel, h=-100)
    return peaks_indices_windowed(f, spectrum['magnitudes'])

# Možno previesť do relatívnych čísel vzhľadom na magnitúdy v okne
n = 16 # len(train_spectra['bins']) // 4
K = np.arange(3, n, 3)
E = np.linspace(0, np.max(train_spectra['magnitudes']), 3)  
H = np.linspace(0, np.max(train_spectra['magnitudes']), 8)

best = grid_search(train_spectra, train_labels, algo_no1, 'macro', K, E, H)
m_train = evaluation(train_spectra['bins'], train_labels, algo_no1(train_spectra, *best), 'macro')
m_test = evaluation(test_spectra['bins'], test_labels, algo_no1(test_spectra, *best), 'macro')

print('k =', ', '.join([f'{x}' for x in K]))
print('e =', ', '.join([f'{x:.2f}' for x in E]))
print('h_rel =', ', '.join([f'{x:.2f}' for x in H]))
print()
print('Best k =', best[0])
print('Best e =', best[1])
print('Best h_rel =', best[2])
print()
print('Train:', m_train)
print('Test:', m_test)

### Úspešnosť: algoritmus prechodu nulou do záporu

In [ ]:
def algo_no2(spectrum, k, s):
    f = partial(find_peaks_zero_crossing, k=int(k), slope=s)
    return peaks_indices_windowed(f, spectrum['magnitudes'])


K = np.arange(1, 50)
S = np.linspace(0, 6, 10)
print('k=', K); print('slope=', S)


best = grid_search(train_spectra, train_labels, algo_no2, 'macro', K, S)
m_train = evaluation(train_spectra['bins'], train_labels, algo_no2(train_spectra, *best), 'macro')
m_test = evaluation(test_spectra['bins'], test_labels, algo_no2(test_spectra, *best), 'macro')

print('k =', ', '.join([f'{x}' for x in K]))
print('s =', ', '.join([f'{x:.2f}' for x in S]))
print()
print('Best k =', best[0])
print('Best s =', best[1])
print()
print('Train:', m_train)
print('Test:', m_test)

### Úspešnosť: algoritmus horského turistu

In [ ]:
def algo_no3(spectrum, t, h, p, i):
    f = partial(find_peaks_hill_walker, tolerance=t, hole=int(h), prominence=p, isolation=int(i))
    return peaks_indices_windowed(f, spectrum['magnitudes'])

H = np.linspace(0, np.max(spectrum['magnitudes']), 6)
L = np.linspace(0, len(spectrum['bins']), 6)
print('tolerance=', H); print('hole=', L); print('prominence=', H); print('isolation=', L)


best = grid_search(train_spectra, train_labels, algo_no3, 'macro', H, L, H, L)
m_train = evaluation(train_spectra['bins'], train_labels, algo_no3(train_spectra, *best), 'macro')
m_test = evaluation(test_spectra['bins'], test_labels, algo_no3(test_spectra, *best), 'macro')

print('h =', ', '.join([f'{x}' for x in K]))
print('l =', ', '.join([f'{x:.2f}' for x in S]))
print()
print('Best tolerance =', best[0])
print('Best hole =', best[1])
print('Best prominence =', best[2])
print('Best isolation =', best[3])
print()
print('Train:', m_train)
print('Test:', m_test)

### Hľadanie špičiek vo vlastných datasetoch

In [ ]:
vibrations, FS_HZ = load_dataset('esp', 'L83_4940_Alexyho_Svantnerova.csv')
vibrations.info()

In [ ]:
a = 20000
b = 26000
t = vibrations.index.array[a:b]
y = vibrations.z.array[a:b]

y = signal_threshold(y - y.mean(), 1)
#t, y = smooth_signal(t, y, boxcar, 32)
plt.rcParams['figure.figsize'] = (20, 4)
plt.plot(t, y)

In [ ]:
spectrum = frequency_spectrum(t, y, fs=FS_HZ, window=hann, Wn=512, overlap=0.5, mode='log')
# welch_method(spectrum, 3)
# smooth_spectra(spectrum, boxcar, 10)

# func = partial(find_peaks_threshold, t=-15)
func = partial(find_peaks_neighbours, k=20, e=0, h_rel=5, h=-100)
# func = partial(find_peaks_zero_crossing, k=30, slope=10)
# func = partial(find_peaks_hill_walker, tolerance=5, hole=0, prominence=10, isolation=5)
peaks = peaks_indices_windowed(func, spectrum['magnitudes'])

plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

### Odčítanie bežiaceho priemeru 

- online aproximácia pre: `y - y.mean()`
- Cumulative moving average: https://en.wikipedia.org/wiki/Moving_average#Cumulative_moving_average
- Welfordov algoritmus (2.20 v BP)

In [ ]:
def filter_running_average(y: np.array):
    avg = y[0]
    n = 1

    for i, sample in enumerate(y[1:], start=1):
        n += 1
        avg += (sample - avg) / n 
        y[i] -= avg
        
    return y

### Demonštrácia pipeline

In [ ]:
# Získanie signálu v časovej doméne
t = vibrations.index.array
y = vibrations.z.array

# Odčítanie bežiaceho priemeru (online aproximácia pre: y - y.mean())
y = filter_running_average(y)

# Orezanie vzoriek s nízkou magnitúdou
y = signal_threshold(y, 1)

# Vyhladenie signálu v času
t, y = smooth_signal(t, y, boxcar, 2)

# Frekvenčná transformácia (fft, rfft, dct-ii, dct-iv)
spectrum = frequency_spectrum(t, y, fs=FS_HZ, window=hann, Wn=256, overlap=0.5, trans='fft', mode='log')

# Vyhadenie spektier
smooth_spectra(spectrum, boxcar, 2)

# Welchova metóda priemerovanie n spektrogramov
welch_method(spectrum, n=2)

# Hľadanie špičiek v spektre
# func = partial(find_peaks_threshold, t=-15)
func = partial(find_peaks_neighbours, k=10, e=0, h_rel=0, h=-100)
# func = partial(find_peaks_zero_crossing, k=5, slope=6)
# func = partial(find_peaks_hill_walker, tolerance=10, hole=1, prominence=8, isolation=5)
peaks = peaks_indices_windowed(func, spectrum['magnitudes'])

# Nakresli spektrogram
print(spectrum['t_windows'].shape)
print(spectrum['bins'].shape)
print(spectrum['magnitudes'].shape)
plot_spectra_heatmap(spectrum, peaks) # ylog=True, yrange=(0,50))

### Vizualizácia detegovaných udalostí

In [ ]:
t = vibrations.index.array
y = vibrations.z.array

spectrum = frequency_spectrum(t, y, fs=FS_HZ, window=hann, Wn=256, overlap=0.5, trans='fft', mode='log')
# smooth_spectra(spectrum, boxcar, 5)

# func = partial(find_peaks_threshold, t=-15)
func = partial(find_peaks_neighbours, k=10, e=0, h_rel=0, h=-100)
# func = partial(find_peaks_zero_crossing, k=5, slope=6)
# func = partial(find_peaks_hill_walker, tolerance=10, hole=1, prominence=8, isolation=5)

peaks = peaks_indices_windowed(func, spectrum['magnitudes'])
print(spectrum['t_windows'].shape)
print(spectrum['bins'].shape)
print(spectrum['magnitudes'].shape)
plot_spectra_heatmap(spectrum, peaks) #ylog=True, yrange=(0,50))

In [ ]:
# Zisti či sa jedná sa o udalosť podľa tolerancie trvania
events = event_detector(spectrum['bins'], spectrum['magnitudes'], peaks, min_duration=8, time_proximity=6)
plot_spectra_events(spectrum, events)

In [ ]:
plot_spectra_event_vs_peaks(spectrum, events, peaks)

In [ ]:
A = partial(find_peaks_threshold, t=-15)
B = partial(find_peaks_neighbours, k=9, e=0, h_rel=10, h=-100)
C = partial(find_peaks_zero_crossing, k=37, slope=5.33)
D = partial(find_peaks_hill_walker, tolerance=10, hole=1, prominence=8, isolation=5)

def dataset_spectrum(filename, algo):
    windows = [64, 128, 256, 512]
    fig, axis = plt.subplots(4, 1, figsize=(20, 30))
    
    ts, FS_HZ = load_dataset('esp', filename)
    t = ts.index.array
    y = ts.z.array

    for i, w in enumerate(windows):
        spectrum = frequency_spectrum(t, y, fs=FS_HZ, window=hann, Wn=w, overlap=0.5, trans='fft', mode='log')
        # smooth_spectra(spectrum, boxcar, 5)
        peaks = peaks_indices_windowed(algo, spectrum['magnitudes'])
        events = event_detector(spectrum['bins'], spectrum['magnitudes'], peaks, min_duration=4, time_proximity=6)
        plot_spectra_event_vs_peaks(spectrum, events, peaks, ax=axis[i])
        #plot_spectra_heatmap(spectrum, ax=axis[i])

In [ ]:
dataset_spectrum('L83_4940_Alexyho_Svantnerova.csv', D)

In [ ]:
dataset_spectrum('L20_3014_Zaluhy-Drobneho.csv', D)

In [ ]:
dataset_spectrum('L35_1915_Borska_Zaluhy.csv', D)